In [1]:
import os
import sqlite3

import numpy as np
import pandas as pd

from tqdm import tqdm
from bisect import bisect
from loguru  import logger

from multiprocessing import Pool, cpu_count, Manager, Process, Value

# pd.set_option('display.max_columns', None)
from check_input import check_db
from preprocessing import build_feature2ndscore, get_db_rid2rn, get_db_rn2fpath, return_pr2tr_id_map, return_nrt_intervel
from database import get_rid2chrom_conn, close_rid2chrom_conn, get_run_native2chrom_fpath
from mrgroup import get_cmrg_messages
from format_data import return_mr_features, initial_format, output_format
from openswath_feature import get_os_features
from discriminate import calc_score_cut, calc_results


debug_mode = False
# 4G
map_size = 36
fdr_precursor = 0.01

nrt_interval_percent = 5e-4
nrt_width_percent = 0.02

n_mrg = 3
min_nuf = 2

n_threads = cpu_count()

seed = 123

db_fpath = "/mnt/data_nas/lyc/project/JointDIA/work/002-MCB/DDALib/10Rawdatas-10Percent-top6-Lib20240131/openswath/test_pp/merged.osw"
chrom_dpath = "/mnt/data_nas/lyc/project/JointDIA/work/002-MCB/DDALib/10Rawdatas-10Percent-top6-Lib20240131/openswath/test_osw"
work_dpath = "/mnt/data_nas/lyc/project/JointAnalysis/work/MCB/MCB_MSF_DDALib_top6_10Percent_Lib20240311/test_jointAnalysis"

In [2]:
map_size = 2 ** map_size
if not os.path.exists(work_dpath):
    os.makedirs(work_dpath)
log_fpath = os.path.join(work_dpath, "JointAnalysis.log")
logger.add(log_fpath, format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}", mode="w")
logger.info(f'JointAnalysis Workflow')

2024-03-29 16:01:03.928 | INFO     | __main__:<module>:6 - JointAnalysis Workflow


In [3]:
logger.info(f'Check db_infile: {db_fpath}')
check_db(db_fpath, logger)

feature2ndscore_fpath = os.path.join(work_dpath, "feature2ndscore.db")
logger.info(f'Save ndscores to db: {feature2ndscore_fpath}')
build_feature2ndscore(db_fpath, feature2ndscore_fpath, map_size)

logger.info(f'Organize the necessary inputs')
rid2rn = get_db_rid2rn(db_fpath)
rid_list = [k for k in rid2rn.keys()]
rn2chrom_fpath = get_db_rn2fpath(chrom_dpath, "sqMass")
pr2tr_id_map = return_pr2tr_id_map(db_fpath)

nrt_intervel, nrt_width = return_nrt_intervel(db_fpath, nrt_interval_percent, nrt_width_percent)

2024-03-29 16:01:05.375 | INFO     | __main__:<module>:1 - Check db_infile: /mnt/data_nas/lyc/project/JointDIA/work/002-MCB/DDALib/10Rawdatas-10Percent-top6-Lib20240131/openswath/test_pp/merged.osw
2024-03-29 16:01:05.676 | INFO     | __main__:<module>:5 - Save ndscores to db: /mnt/data_nas/lyc/project/JointAnalysis/work/MCB/MCB_MSF_DDALib_top6_10Percent_Lib20240311/test_jointAnalysis/feature2ndscore.db
2024-03-29 16:01:55.821 | INFO     | __main__:<module>:8 - Organize the necessary inputs


In [4]:
logger.info(f'Save nativeID2chromID')
rid2chrom_conn = get_rid2chrom_conn(rid2rn, rn2chrom_fpath)
rid_native2chromid_fpath = get_run_native2chrom_fpath(rid2chrom_conn, work_dpath, map_size / 8)
close_rid2chrom_conn(rid2chrom_conn)

2024-03-29 16:06:24.629 | INFO     | __main__:<module>:1 - Save nativeID2chromID


In [5]:
logger.info(f'Get MRGroup')
m_conn = sqlite3.connect(db_fpath)
m_cur = m_conn.cursor()
m_cur.execute(f'SELECT ID FROM PRECURSOR')

precursor_ids = np.array(m_cur.fetchall()).squeeze()
m_cur.close()
m_conn.close()

num_precursor = precursor_ids.shape[0]
logger_n = 10 ** (len(str(num_precursor)) - 2)
n_precur = num_precursor // n_threads
precurs_list = [precursor_ids[i * n_precur : (i + 1) * n_precur].tolist() for i in range(n_threads)]
_ = [precurs_list[i].append(precursor_ids[i + n_precur * n_threads]) for i in range(len(precursor_ids) - n_precur * n_threads)]

results_collector = Manager().list()
counter = Manager().Value('d',0)
logger.info(f"( {counter.value} / {num_precursor}) precursor has Calculated...")
extractors = []
for precur_ids in precurs_list:
    p = Process(target = get_cmrg_messages, 
                args =  (precur_ids, db_fpath, feature2ndscore_fpath, rid_native2chromid_fpath,
                            pr2tr_id_map, rid2rn, rn2chrom_fpath, nrt_intervel, nrt_width,
                            n_mrg, min_nuf, logger_n, debug_mode, results_collector, counter, num_precursor, logger, ))
    p.daemon = True
    extractors.append(p)
    p.start()
for p in extractors:
    p.join()

logger.info(f'Get MR features')
mr_iter_features = return_mr_features(results_collector)
del results_collector, counter

logger.info(f'Get OS features')
target_fids = mr_iter_features["FEATURE_ID"].values
os_feature_pd = get_os_features(target_fids, db_fpath)



2024-03-29 16:09:27.069 | INFO     | __main__:<module>:1 - Get MRGroup
2024-03-29 16:09:28.728 | INFO     | __main__:<module>:18 - ( 0 / 243530) precursor has Calculated...
2024-03-29 16:13:46.578 | INFO     | mrgroup:get_cmrg_messages:253 - ( 10000 / 243530) precursor has Calculated...
2024-03-29 16:13:46.689 | INFO     | mrgroup:get_cmrg_messages:253 - ( 10000 / 243530) precursor has Calculated...
2024-03-29 16:16:17.826 | INFO     | mrgroup:get_cmrg_messages:253 - ( 20000 / 243530) precursor has Calculated...
2024-03-29 16:16:17.901 | INFO     | mrgroup:get_cmrg_messages:253 - ( 20000 / 243530) precursor has Calculated...
2024-03-29 16:18:47.508 | INFO     | mrgroup:get_cmrg_messages:253 - ( 30000 / 243530) precursor has Calculated...
2024-03-29 16:21:30.925 | INFO     | mrgroup:get_cmrg_messages:253 - ( 40000 / 243530) precursor has Calculated...
2024-03-29 16:24:18.763 | INFO     | mrgroup:get_cmrg_messages:253 - ( 50000 / 243530) precursor has Calculated...
2024-03-29 16:28:58.29

KeyError: 'RIGHT_WIDTH'

In [9]:
logger.info(f'Initial format')
mr_iter_features = initial_format(db_fpath, mr_iter_features, os_feature_pd)

logger.info(f'Discriminate')
ignored_columns = ["PRECURSOR_ID", "FEATURE_ID", "decoy", "run_id",  "leftWidth", "rightWidth", "Intensity", "aggr_prec_Peak_Area", "aggr_prec_Peak_Apex", "MRG_RANK"]
iter_mr_columns = [col for col in mr_iter_features.columns if col not in ignored_columns]

mr_iter_res = calc_results(scored_columns = iter_mr_columns,
                            initial_column = "MRG_RANK",
                            initial_ascending = True, 
                            data_pd = mr_iter_features,
                            n_threads = n_threads,
                            seed = seed)

2024-03-29 17:26:02.330 | INFO     | __main__:<module>:1 - Initial format
2024-03-29 17:28:50.807 | INFO     | __main__:<module>:4 - Discriminate
/home/lyc/anaconda3/envs/dreamdia/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/lyc/anaconda3/envs/dreamdia/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:31:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:34:12] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:36:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:38:46] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: 

In [12]:
import sqlite3

def return_precid2pepid(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    precid2pepid = pd.read_sql(f'SELECT PRECURSOR_ID, PEPTIDE_ID FROM PRECURSOR_PEPTIDE_MAPPING', conn)
    conn.close()
    precid2pepid = precid2pepid.set_index("PRECURSOR_ID")["PEPTIDE_ID"].to_dict()
    data["PEPTIDE_ID"] = data["PRECURSOR_ID"].apply(lambda x : precid2pepid[x])
    return data

def return_pepid2protid(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    pepid2protid = pd.read_sql(f'SELECT PEPTIDE_ID, PROTEIN_ID FROM PEPTIDE_PROTEIN_MAPPING', conn)
    conn.close()
    pepid2protid = pepid2protid.set_index("PEPTIDE_ID")["PROTEIN_ID"].to_dict()
    data["PROTEIN_ID"] = data["PEPTIDE_ID"].apply(lambda x : pepid2protid[x])
    return data

def return_protid2prot(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    protid2prot = pd.read_sql(f'SELECT ID, PROTEIN_ACCESSION FROM PROTEIN', conn)
    conn.close()
    protid2prot = protid2prot.set_index("ID")["PROTEIN_ACCESSION"].to_dict()
    data["ProteinName"]  = data["PROTEIN_ID"].apply(lambda x : protid2prot[x])
    return data

# def return_precid2grouplabel(db_fpath, data):
#     conn = sqlite3.connect(db_fpath)
#     precid2grouplabel = pd.read_sql(f'SELECT ID, GROUP_LABEL FROM PRECURSOR', conn)
#     conn.close()
#     precid2grouplabel = precid2grouplabel.set_index("ID")["GROUP_LABEL"].to_dict()
#     data["GROUP_LABEL"] = data["PRECURSOR_ID"].apply(lambda x : precid2grouplabel[x])
#     return data

def return_precid2pmz(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    precid2pmz = pd.read_sql(f'SELECT ID, PRECURSOR_MZ FROM PRECURSOR', conn)
    conn.close()
    precid2pmz = precid2pmz.set_index("ID")["PRECURSOR_MZ"].to_dict()
    data["mz"] = data["PRECURSOR_ID"].apply(lambda x : precid2pmz[x])
    return data

def return_precid2pcharge(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    precid2pcharge = pd.read_sql(f'SELECT ID, CHARGE FROM PRECURSOR', conn)
    conn.close()
    precid2pcharge = precid2pcharge.set_index("ID")["CHARGE"].to_dict()
    data["Charge"] = data["PRECURSOR_ID"].apply(lambda x : precid2pcharge[x])
    return data

def return_runid2filename(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    runid2filename = pd.read_sql(f'SELECT ID, FILENAME FROM RUN', conn)
    conn.close()
    runid2filename = runid2filename.set_index("ID")["FILENAME"].to_dict()
    data["filename"] = data["run_id"].apply(lambda x : runid2filename[x])
    return data

def return_featid2irt(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    featid2irt = pd.read_sql(f'SELECT ID, NORM_RT FROM FEATURE', conn)
    conn.close()
    featid2irt = featid2irt.set_index("ID")["NORM_RT"].to_dict()
    data["iRT"] = data["FEATURE_ID"].apply(lambda x : featid2irt[x])
    return data

def return_pepid2seq(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    pepid2seq = pd.read_sql(f'SELECT ID, UNMODIFIED_SEQUENCE FROM PEPTIDE', conn)
    conn.close()
    pepid2seq = pepid2seq.set_index("ID")["UNMODIFIED_SEQUENCE"].to_dict()
    data["Sequence"] = data["PEPTIDE_ID"].apply(lambda x : pepid2seq[x])
    return data

def return_pepid2mseq(db_fpath, data):
    conn = sqlite3.connect(db_fpath)
    pepid2mseq = pd.read_sql(f'SELECT ID, MODIFIED_SEQUENCE FROM PEPTIDE', conn)
    conn.close()
    pepid2mseq = pepid2mseq.set_index("ID")["MODIFIED_SEQUENCE"].to_dict()
    data["FullPeptideName"] = data["PEPTIDE_ID"].apply(lambda x : pepid2mseq[x])
    return data

def output_format1(db_fpath, mr_iter_res):
    mr_iter_res = mr_iter_res.reset_index()
    mr_iter_res = return_precid2pepid(db_fpath, mr_iter_res)
    mr_iter_res = return_pepid2protid(db_fpath, mr_iter_res)
    mr_iter_res = return_protid2prot(db_fpath, mr_iter_res)
    # mr_iter_res = return_precid2grouplabel(db_fpath, mr_iter_res)
    mr_iter_res = return_precid2pmz(db_fpath, mr_iter_res)
    mr_iter_res = return_precid2pcharge(db_fpath, mr_iter_res)
    mr_iter_res = return_runid2filename(db_fpath, mr_iter_res)
    mr_iter_res = return_featid2irt(db_fpath, mr_iter_res)
    mr_iter_res = return_pepid2seq(db_fpath, mr_iter_res)
    mr_iter_res = return_pepid2mseq(db_fpath, mr_iter_res)
    mr_iter_res["assay_rt"] = mr_iter_res["RT"] - mr_iter_res["delta_rt"]
    

    scored_columns = []
    bases_columns = []
    for col in mr_iter_res.columns:
        if col.startswith("VAR_") or col.startswith("MR_"):
            scored_columns.append(col)
        else:
            bases_columns.append(col)

    mr_iter_res = mr_iter_res.loc[:, bases_columns]
    return mr_iter_res

In [34]:
 results = output_format1(db_fpath, mr_iter_res)

trans = []
for _, (mseq, charge, decoy) in enumerate(zip(results["FullPeptideName"].values, results["Charge"].values, results["decoy"].values)):
    if decoy == 0:
        trans.append(mseq + "_" + str(charge))
    else:
        trans.append("DECOY_" + mseq + "_" + str(charge))
results["transition_group_id"] = trans
results_format = results.loc[:, ["transition_group_id",
                                 "decoy",
                                 "run_id",
                                 "filename", 
                                 "RT",
                                 "assay_rt",
                                 "delta_rt",
                                 "iRT",
                                 "Sequence",
                                 "FullPeptideName",
                                 "Charge",
                                 "mz",
                                 "Intensity",
                                 "aggr_prec_Peak_Area",
                                 "aggr_prec_Peak_Apex",
                                 "leftWidth",
                                 "rightWidth",
                                 "ProteinName",
                                 "jd_score"]]

In [37]:
results_format.head(4)

,transition_group_id,decoy,run_id,filename,RT,assay_rt,delta_rt,iRT,Sequence,FullPeptideName,Charge,mz,Intensity,aggr_prec_Peak_Area,aggr_prec_Peak_Apex,leftWidth,rightWidth,ProteinName,jd_score
0,.(UniMod:1)AAAAAAAAAAAAGDSDSWDADTFSMEDPVRK_3,0,7979900529762267651,/mnt/data_nas/lyc/project/JointDIA/data/MCB/BG...,6626.10,6681.5413,-55.4413,82.9356,AAAAAAAAAAAAGDSDSWDADTFSMEDPVRK,.(UniMod:1)AAAAAAAAAAAAGDSDSWDADTFSMEDPVRK,3,1018.124485,5947190.0,4.173890e+06,1.751130e+06,6601.324707,6638.442383,sp|Q66JS6|EI3JB_MOUSE,0.961575
1,.(UniMod:1)AAAAAAAAAAGAAGGR_2,0,7979900529762267651,/mnt/data_nas/lyc/project/JointDIA/data/MCB/BG...,4557.35,4606.7990,-49.4490,58.1088,AAAAAAAAAAGAAGGR,.(UniMod:1)AAAAAAAAAAGAAGGR,2,620.823278,24966500.0,3.115621e+08,7.817065e+07,4540.854492,4573.848633,sp|Q8CCS6|PABP2_MOUSE,0.999886
2,.(UniMod:1)AAAAAAAGGAALAVSTGLETATLQK_2,0,7979900529762267651,/mnt/data_nas/lyc/project/JointDIA/data/MCB/BG...,7360.27,7421.6273,-61.3573,91.7463,AAAAAAAGGAALAVSTGLETATLQK,.(UniMod:1)AAAAAAAGGAALAVSTGLETATLQK,2,1114.100071,4590070.0,5.403477e+06,1.948072e+06,7344.550781,7378.718262,sp|Q9CQ25|MZT2_MOUSE,0.996836
3,.(UniMod:1)AAAAAAAGGAALAVSTGLETATLQK_3,0,7979900529762267651,/mnt/data_nas/lyc/project/JointDIA/data/MCB/BG...,7360.34,7420.5903,-60.2503,91.7471,AAAAAAAGGAALAVSTGLETATLQK,.(UniMod:1)AAAAAAAGGAALAVSTGLETATLQK,3,743.069139,1466010.0,1.120476e+06,8.125468e+05,7347.208008,7374.554199,sp|Q9CQ25|MZT2_MOUSE,0.999590


In [39]:
from reports import stats

In [40]:
results_format = stats(results_format, "jd_score", logger)